In [1]:
!pip install numpy


In [2]:



import zipfile
import os

# Unzip the dataset
zip_path = "/content/pda-159-dataset.zip"
extract_path = "/content/"
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

print("Dataset extracted successfully!")

Dataset extracted successfully!


In [3]:

from Cache import Cache
from DataLoader import DataLoaderPintos


In [4]:
dataloader = DataLoaderPintos(["pda-159-dataset/filesys/extended/dir-open.csv"])

In [5]:
from ReflexAgent import LRUAgent, LFUAgent, MRUAgent

In [6]:
env = Cache(dataloader, 50,
            feature_selection=('Base',),
            reward_params=dict(name='our', alpha=0.5, psi=10, mu=1, beta=0.3),
            allow_skip=False)

In [7]:

agents = {
    'LRU': LRUAgent(env.n_actions),
    'LFU': LFUAgent(env.n_actions),
    'MRU': MRUAgent(env.n_actions)
}

miss_rates = {}

for name, agent in agents.items():
    print(f"-------------------- {name} --------------------")

    # Reset environment
    observation = env.reset()

# Run simulation
    while not env.hasDone():
        action = agent.choose_action(observation)
        observation_, reward = env.step(action)
        observation = observation_

    # Record miss rate
    miss_rate = env.miss_rate()
    miss_rates[name] = miss_rate

    print(f"Agent={name}: Miss Rate={miss_rate}")

# Compare miss rates
best_agent = min(miss_rates, key=miss_rates.get)
print(f"Best Agent: {best_agent} with Miss Rate={miss_rates[best_agent]}")


-------------------- LRU --------------------
Agent=LRU: Miss Rate=0.1994514106583072
-------------------- LFU --------------------
Agent=LFU: Miss Rate=0.1841692789968652
-------------------- MRU --------------------
Agent=MRU: Miss Rate=0.4647335423197492
Best Agent: LFU with Miss Rate=0.1841692789968652


In [12]:
from DQNAgent import DQNAgent
from Cache import Cache
from DataLoader import DataLoaderPintos
data_loader = DataLoaderPintos("pda-159-dataset/filesys/extended/dir-open.csv", boot=False)  # or your own file

env = Cache(
    requests=data_loader,
    cache_size=100,
    feature_selection=('Base','Age','Frequency'),
    reward_params=dict(name='our', alpha=0.5, psi=10, mu=1, beta=0.3)
)

# Initialize DQN agent
dqn_agent = DQNAgent(
    n_actions=env.n_actions,
    n_features=len(env.reset()['features']),
    learning_rate=0.005,
    reward_decay=0.9,
    e_greedy_min=(0.05, 0.1),
    e_greedy_max=(0.2, 0.3),
    e_greedy_init=(0.1, 0.2),
    e_greedy_increment=(0.01, 0.01),
    e_greedy_decrement=(0.005, 0.005),
    reward_threshold=5,
    explore_mentor='LRU',
    replace_target_iter=200,
    memory_size=10000,
    batch_size=32,
    output_graph=False,
    verbose=1
)

# Train the DQN agent over multiple episodes
episodes = 10
for ep in range(episodes):
    observation = env.reset()
    while not env.hasDone():
        action = dqn_agent.choose_action(observation)
        observation_, reward = env.step(action)
        dqn_agent.store_transition(observation, action, reward, observation_)
        dqn_agent.learn()
        observation = observation_

# Final evaluation after training
final_observation = env.reset()
while not env.hasDone():
    action = dqn_agent.choose_action(final_observation)
    final_observation, _ = env.step(action)

# Print final miss rate
print("\n================== Final Evaluation ==================")
print(f"DQN Miss Rate after {episodes} episodes: {env.miss_rate():.6f}")


================== Final Evaluation ==================
DQN Miss Rate after 10 episodes: 0.141066
